In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp arc

In [ ]:
#| export
import json, os
import numpy as np
import gym
from gym import spaces
from time import sleep
import pygame
from matplotlib import colors       
# import copy

In [ ]:
#| export


class ARCDataProcessor:
    def __init__(self, config_dict, arc_dict):
        self.arc_dict = arc_dict
        self.grid_shape = self.determine_grid_shape()
        
        if 'grid_shape' in config_dict:
            self.grid_shape = config_dict.get('grid_shape')
        
        self.input_set = config_dict.get('input_set', None)
        if self.input_set not in {'env_only', 'inputs_only', 'both', None}:
            raise ValueError("input_set must be 'env_only', 'inputs_only', 'both', or None")
        
        self.action_set = config_dict.get('action_set', None)
        if self.action_set not in {'dims_only', None}:
            raise ValueError("action_set must be 'dims_only' or None")
        
        self.index = config_dict.get('index', 0)
        self.initial_index = self.index if 'index' in config_dict else None
        self.output_set = config_dict.get('output_set', True)
        self.dataset = config_dict.get('dataset', None)

        if self.dataset is None:
            raise ValueError("dataset must be provided in config_dict")

        if self.action_set == 'dims_only':
            if self.grid_shape is None:
                raise ValueError("grid_shape cannot be None when action_set is 'dims_only'")
            if self.input_set is not None:
                raise ValueError("input_set must be None when action_set is 'dims_only'")

        self.create_env()
        self.info = self.create_info()

    def add_rows(self, num_rows):
        num_rows = round(num_rows)
        self.env = np.pad(self.env, ((0, num_rows), (0, 0)), mode='constant', constant_values=0)

    def remove_rows(self, num_rows):
        num_rows = round(num_rows)
        if num_rows >= self.env.shape[0]:
            num_rows = self.env.shape[0] - 1
        self.env = self.env[:-num_rows, :] if self.env.shape[0] > num_rows else self.env

    def add_columns(self, num_columns):
        num_columns = round(num_columns)
        self.env = np.pad(self.env, ((0, 0), (0, num_columns)), mode='constant', constant_values=0)

    def remove_columns(self, num_columns):
        num_columns = round(num_columns)
        if num_columns >= self.env.shape[1]:
            num_columns = self.env.shape[1] - 1
        self.env = self.env[:, :-num_columns] if self.env.shape[1] > num_columns else self.env

    def process_remaining_values(self, values):
        for i, value in enumerate(values):
            row, col = divmod(i, self.env.shape[1])
            if row < self.env.shape[0] and col < self.env.shape[1]:
                self.env[row, col] = value

    def get_array(self, key, index, sub_key):
        return np.array(self.arc_dict[key][index][sub_key])

    def next(self):
        if self.index >= len(self.arc_dict[self.dataset]):
            return False
        if self.initial_index is not None:
            return False
        self.index += 1
        self.create_env()
        return True

    def reset(self):
        if self.initial_index is None:
            self.index = 0
        else:
            self.index = self.initial_index
        self.create_env()

    def create_env(self):
        self.env = np.array(self.arc_dict[self.dataset][self.index]['input'])

    def process_dimensions(self, actions):
        if len(actions) == 2:
            num_rows, num_cols = actions
        elif len(actions) == 1:
            num_rows = num_cols = actions[0]
        else:
            raise ValueError("Actions must have one or two values")

        if num_rows > 0:
            self.add_rows(num_rows)
        elif num_rows < 0:
            self.remove_rows(abs(num_rows))

        if num_cols > 0:
            self.add_columns(num_cols)
        elif num_cols < 0:
            self.remove_columns(abs(num_cols))

    def get_input_dimensions(self):
        return np.array(self.arc_dict[self.dataset][self.index]['input']).shape

    def get_output_dimensions(self):
        return np.array(self.arc_dict[self.dataset][self.index]['output']).shape

    def get_env_dimensions(self):
        return self.env.shape

    def apply_actions(self, actions):
        value_index = 0
        if self.grid_shape == 'equal':
            self.process_dimensions(actions[:1])
            value_index = 1
        elif self.grid_shape == 'unequal':
            self.process_dimensions(actions[:2])
            value_index = 2
        
        if self.action_set != 'dims_only':
            self.process_remaining_values(actions[value_index:])

    def create_info(self):
        info = {}
        info['dims'] = 0
        info['num_actions'] = 0
        info['grid_shape'] = self.grid_shape

        if self.grid_shape == 'equal':
            info['num_actions'] = 1
            info['dims'] = 3
        elif self.grid_shape == 'unequal':
            info['num_actions'] = 2
            info['dims'] = 6

        if self.action_set != 'dims_only':
            if self.input_set in {'env_only', 'both'}:
                flattened_env = self.env.flatten()
                info['env'] = len(flattened_env)
                info['num_actions'] += len(flattened_env)
            
            if self.input_set in {'inputs_only', 'both'}:
                flattened_input = self.get_array(self.dataset, self.index, 'input').flatten()
                info['inputs'] = len(flattened_input)

            flattened_output = self.get_array(self.dataset, self.index, 'output').flatten()
            info['outputs'] = len(flattened_output)

        return info

    def get_state(self):
        values = {}
        self.info['num_actions'] = 0
        if self.grid_shape == 'equal':
            self.info['num_actions'] = 1
            values['env_dims'] = (self.get_env_dimensions()[1],)
            values['input_dims'] = (self.get_input_dimensions()[1],)
            values['output_dims'] = (self.get_output_dimensions()[1],)
        elif self.grid_shape == 'unequal':
            self.info['num_actions'] = 2
            values['env_dims'] = self.get_env_dimensions()
            values['input_dims'] = self.get_input_dimensions()
            values['output_dims'] = self.get_output_dimensions()

        if self.action_set != 'dims_only':
            if self.input_set in {'env_only', 'both'}:
                values['env'] = self.env
                self.info['env'] = self.env.size
                self.info['num_actions'] += self.env.size
            
            if self.input_set in {'inputs_only', 'both'}:
                values['input'] = self.get_array(self.dataset, self.index, 'input')
            
            if self.output_set:
                values['output'] = self.get_array(self.dataset, self.index, 'output')

        return values, self.info

    def fitness_function_arrays(self, output_array, env_array):
        # First metric: square of the difference between the dimensions
        dim_metric = (env_array.shape[0] - output_array.shape[0]) ** 2 + (env_array.shape[1] - output_array.shape[1]) ** 2

        # Second metric: square of the difference between each element in the arrays
        element_metric = 0
        for i in range(max(env_array.shape[0], output_array.shape[0])):
            for j in range(max(env_array.shape[1], output_array.shape[1])):
                env_value = env_array[i, j] if i < env_array.shape[0] and j < env_array.shape[1] else None
                output_value = output_array[i, j] if i < output_array.shape[0] and j < output_array.shape[1] else None
                if env_value is None or output_value is None:
                    element_metric += 25
                else:
                    element_metric += (env_value - output_value) ** 2

        # Final metric
        if self.action_set == 'dims_only':
            final_metric = dim_metric
        else:
            final_metric = dim_metric + element_metric

        return final_metric

    def fitness_function(self):
        output_array = np.array(self.arc_dict[self.dataset][self.index]['output'])
        env_array = self.env
        return self.fitness_function_arrays(output_array, env_array)

    def get_input(self, dataset):
        return np.array(self.arc_dict[dataset][self.index]['input'])

    def get_output(self, dataset):
        return np.array(self.arc_dict[dataset][self.index]['output'])

    def get_info(self):
        return self.info

    def determine_grid_shape(self):
        output_dims = [np.array(task['output']).shape for task in self.arc_dict['train']]
        input_dims = [np.array(task['input']).shape for task in self.arc_dict['train']]

        if len(set(output_dims)) == 1:
            if len(set(input_dims)) == 1 and output_dims[0] == input_dims[0]:
                return None
            return 'equal'
        return 'unequal'

    def get_env_inputs_names(self):
        input_names = []
        if 'dims' in self.info:
            if self.info['dims'] == 2:
                input_names += ['IWE', 'IWO']
            elif self.info['dims'] == 3:
                input_names += ['IWE', 'IWI', 'IWO']
            elif self.info['dims'] == 4:
                input_names += ['IWE', 'IHE', 'IWO', 'IHO']
            elif self.info['dims'] == 6:
                input_names += ['IWE', 'IHE', 'IWI', 'IHI', 'IWO', 'IHO']
            
        if 'env' in self.info:
            num = self.info['env']
            for i in range(num):
                input_names.append(f'IE{i+1:03}')
            
        if 'inputs' in self.info:
            num = self.info['inputs']
            for i in range(num):
                input_names.append(f'II{i+1:03}')

        if 'outputs' in self.info:
            num = self.info['outputs']
            for i in range(num):
                input_names.append(f'IO{i+1:03}')
            
        return input_names

    def get_env_inputs_indexes(self):
        ninputs = 0

        if 'dims' in self.info:
            ninputs += self.info['dims']

        if 'env' in self.info:
            ninputs += self.info['env']

        if 'inputs' in self.info:
            ninputs += self.info['inputs']

        if 'outputs' in self.info:
            ninputs += self.info['outputs']

        env_inputs_indexes = [i for i in range(ninputs)]

        return env_inputs_indexes

    def get_env_array(self):
        return self.env



In [ ]:

# Example usage:
config_dict = {
    # 'grid_shape': 'unequal',
    'grid_shape': 'equal',
    # 'grid_shape': None,
    # 'input_set': 'env_only',
    # 'input_set': 'both',
    # 'input_set': None,
    'action_set': 'dims_only',
    'dataset': 'train',
    'index': 0
}

arc_dict = {
    'test': [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]]}],
    'train': [
        {'input': [[0, 7, 7], [7, 7, 7], [0, 7, 7]], 'output': [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]}
        # Add more entries as needed
    ]
}

gp = ARCDataProcessor(config_dict, arc_dict)
print('grid_shape', gp.grid_shape)
info = gp.create_info()
print(info)
ins = gp.get_env_inputs_names()
print(ins)

inds = gp.get_env_inputs_indexes()
# print(len(inds))
print(inds)

values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

actions = [-5]
gp.apply_actions(actions)
values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

actions = [-5]
gp.apply_actions(actions)
values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

# actions = [5,4]
# gp.apply_actions(actions)
# values, info = gp.get_state()
# print(info)
# print(len(values))
# print(values)
# print('fitness', gp.fitness_function())


grid_shape equal
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
['IWE', 'IWI', 'IWO']
[0, 1, 2]
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (3,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 72
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (1,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 128
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (0,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 162


In [ ]:
#| export

class ARCEnv(gym.Env):
    def __init__(self):
        super(ARCEnv, self).__init__()
        self.index = 0
        self.env = None
        self.dimensions = []
        self._fitness = 10000  # Initialize fitness to 10000
        self.state = []
        self.done = False

        # Class variables
        self.dataset = None

        # Render settings
        self.screen_width = 1000
        self.screen_height = 500
        self.cell_size = 30
        self.left_pad = 20
        self.height_pad = 20
        self.grid_down = 50
        self.symbol_down = 150
        self.screen = None
        self.isopen = True

        self.cmap = colors.ListedColormap(['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
                                           '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
        self.norm = colors.Normalize(vmin=0, vmax=9)

        self.fitness_label_font_size = 20
        self.fitness_value_font_size = 50

        self.iteration = 1  # Initialize iteration to 1
        self.code = ""  # Example property, ensure to set self.code elsewhere in your class
        self.data = None  # Placeholder for data
        self.arc_data = None  # Placeholder for ARCDataProcessor
        self.num_actions = 0  # Initialize num_actions

    def initialise(self, properties, arc_dict):
        """
        Initialize the environment with properties and arc_dict.
        """
        self.dataset = properties.get('dataset', None)
        self.arc_data = ARCDataProcessor(properties, arc_dict)
        self.reset()

    def step(self, actions):
        """
        Take a step in the environment.
        """
        self.arc_data.apply_actions(actions)

        if self.dataset == 'train':
            self.fitness = self.arc_data.fitness_function()

        self.state, self.info = self.arc_data.get_state()
        
        self.iteration += 1  # Increment iteration
        return self.state, self.fitness, self.done, self.info

    def reset(self):
        """
        Reset the environment to the initial state.
        """
        self.arc_data.reset()
        if self.dataset == 'train':
            self.fitness = self.arc_data.fitness_function()
        else:
            self.fitness = 10000  # or some default value
        self.done = False
        self.state, self.info = self.arc_data.get_state()
        self.iteration = 1  # Reset iteration
        self.num_actions = self.info['num_actions']  # Set num_actions

    def next(self):
        """
        Move to the next state in arc_dict.
        """
        return self.arc_data.next()

    def get_num_actions(self):
        """
        Get the number of actions.
        """
        return self.num_actions

    def get_env_inputs_names(self):
        """
        Get the environment input names.
        """
        return self.arc_data.get_env_inputs_names()

    def get_env_inputs_indexes(self):
        """
        Get the environment input indexes.
        """
        return self.arc_data.get_env_inputs_indexes()

    def set_dataset(self, dataset):
        """
        Set the dataset and update the value in arc_data.
        """
        self.dataset = dataset
        self.arc_data.set_dataset(dataset)

    def call_fitness_function_arrays(self):
        """
        Call fitness_function_arrays on arc_data.
        """
        return self.arc_data.fitness_function_arrays()

    def get_env_array(self):
        """
        Get the environment array from arc_data.
        """
        return self.arc_data.get_env_array()

    @property
    def fitness(self):
        """
        Get the current fitness value.
        """
        return self._fitness

    @fitness.setter
    def fitness(self, value):
        """
        Set the current fitness value.
        """
        self._fitness = value

    def render(self, mode='human'):
        """
        Render the environment using Pygame.
        """
        def draw_grid(screen, grid, top_left_x, top_left_y, cell_size):
            for i, row in enumerate(grid):
                for j, value in enumerate(row):
                    if top_left_x + j * cell_size < self.screen_width and top_left_y + i * cell_size < self.screen_height:
                        color = self.cmap(self.norm(value))[:3]  # Get RGB only, excluding alpha
                        color = tuple(int(c * 255) for c in color)  # Multiply each element by 255
                        pygame.draw.rect(screen, color, (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size))
                        pygame.draw.rect(screen, (255, 255, 255), (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size), 1)
            # Draw a black line around the grid
            if top_left_x < self.screen_width and top_left_y < self.screen_height:
                pygame.draw.rect(screen, (0, 0, 0), (top_left_x, top_left_y, cell_size * len(grid[0]), cell_size * len(grid)), 2)

        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.display.set_caption('ARC Environment')

        input_grid = np.array(self.arc_data.get_input(self.dataset))
        output_grid = np.array(self.arc_data.get_output(self.dataset))
        env_grid = self.arc_data.env

        # Top left coordinates
        input_grid_x = self.left_pad
        input_grid_y = self.grid_down
        arrow_img_x = input_grid_x + input_grid.shape[1] * self.cell_size + self.left_pad
        arrow_img_y = self.symbol_down
        output_grid_x = arrow_img_x + 50 + self.left_pad
        output_grid_y = self.grid_down
        equals_img_x = output_grid_x + output_grid.shape[1] * self.cell_size + self.left_pad
        equals_img_y = self.symbol_down
        env_grid_x = equals_img_x + 50 + self.left_pad
        env_grid_y = self.grid_down
        fitness_text_x = env_grid_x + env_grid.shape[1] * self.cell_size + self.left_pad
        fitness_text_y = self.grid_down + self.height_pad
        fitness_value_y = fitness_text_y + self.fitness_label_font_size + self.height_pad
        tick_cross_y = fitness_value_y + self.fitness_value_font_size + self.height_pad
        table_y = self.grid_down + max(input_grid.shape[0], output_grid.shape[0], env_grid.shape[0]) * self.cell_size + self.height_pad

        # Adjust screen size if necessary
        if fitness_text_x + self.fitness_value_font_size > self.screen_width:
            self.screen_width = fitness_text_x + self.fitness_value_font_size + self.left_pad
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        if env_grid_y + env_grid.shape[0] * self.cell_size > self.screen_height:
            self.screen_height = env_grid_y + env_grid.shape[0] * self.cell_size + self.height_pad
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))

        self.screen.fill((255, 255, 255))  # Clear the screen

        draw_grid(self.screen, input_grid, input_grid_x, input_grid_y, self.cell_size)
        draw_grid(self.screen, output_grid, output_grid_x, output_grid_y, self.cell_size)
        draw_grid(self.screen, env_grid, env_grid_x, env_grid_y, self.cell_size)

        # Load and scale images
        arrow_img = pygame.transform.scale(pygame.image.load('images/arrow.png'), (50, 50))
        equals_img = pygame.transform.scale(pygame.image.load('images/equals.jpg'), (50, 50))
        green_tick_img = pygame.transform.scale(pygame.image.load('images/green_tick.png'), (50, 50))
        red_cross_img = pygame.transform.scale(pygame.image.load('images/red-cross.png'), (50, 50))

        if arrow_img_x < self.screen_width and arrow_img_y < self.screen_height:
            self.screen.blit(arrow_img, (arrow_img_x, arrow_img_y))
        if equals_img_x < self.screen_width and equals_img_y < self.screen_height:
            self.screen.blit(equals_img, (equals_img_x, equals_img_y))

        # Display fitness text
        label_font = pygame.font.Font(None, self.fitness_label_font_size)
        value_font = pygame.font.Font(None, self.fitness_value_font_size)
        label_font.set_bold(True)
        fitness_label = label_font.render("Fitness:", True, (0, 0, 0))
        fitness_value = value_font.render(f"{self.fitness:.2f}", True, (0, 0, 0))
        if fitness_text_x < self.screen_width and fitness_text_y < self.screen_height:
            self.screen.blit(fitness_label, (fitness_text_x, fitness_text_y))
        if fitness_text_x < self.screen_width and fitness_value_y < self.screen_height:
            self.screen.blit(fitness_value, (fitness_text_x, fitness_value_y))

        if self.fitness < 1e-6:
            if fitness_text_x < self.screen_width and tick_cross_y < self.screen_height:
                self.screen.blit(green_tick_img, (fitness_text_x, tick_cross_y))
        else:
            if fitness_text_x < self.screen_width and tick_cross_y < self.screen_height:
                self.screen.blit(red_cross_img, (fitness_text_x, tick_cross_y))

        # Draw table with Code, Iteration, and Index
        table_font = pygame.font.Font(None, 24)
        table_labels = ["Code", "Iteration", "Index"]
        table_values = [os.path.splitext(self.code)[0], self.iteration, self.arc_data.index]  # Display code without file extension
        table_x = self.left_pad
        table_y = table_y

        for i, (label, value) in enumerate(zip(table_labels, table_values)):
            label_surface = table_font.render(label, True, (0, 0, 0))
            value_surface = table_font.render(str(value), True, (0, 0, 0))
            self.screen.blit(label_surface, (table_x + i * 200, table_y))
            self.screen.blit(value_surface, (table_x + i * 200, table_y + 30))

        pygame.display.flip()

    def close(self):
        """
        Close the environment, save the screen to an HTML and an image file.
        """
        if self.screen is not None:
            os.makedirs("c:/tmp/arc/", exist_ok=True)
            pygame.image.save(self.screen, "c:/tmp/arc/screen_image.png")
            
            # Save the screen image to an HTML format
            with open("c:/tmp/arc/screen_image.html", "w") as f:
                f.write(f"<html><body><img src='screen_image.png'></body></html>")

            pygame.display.quit()
            pygame.quit()
            self.isopen = False



In [ ]:
#| gui
# Example usage:
props = {'dir': 'C:\\packages\\arc-prize-2024\\training', 'code':'1_007bbfb7.dat', 'action_set': 'dims_only', 'dataset': 'train'}
file_name = os.path.join(props['dir'], props['code'])
with open(file_name, 'r') as f:
    data = json.load(f)

# arc_dict={}
# arc_dict['data'] = data
arc_env = ARCEnv()
arc_env.initialise(props, data)
arc_env.render()
#    print(state, fitness, done)
# print(arc_env.dimensions)
for i in range(6):
    state, fitness, done, info = arc_env.step([1, 1, 1, 2, 3, 4, 5, 6, 7])
    # print(state, fitness, done)
    print(state, fitness, done, info)
    arc_env.render()
    sleep(1)



{'env_dims': (4,), 'input_dims': (3,), 'output_dims': (9,)} 50 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (5,), 'input_dims': (3,), 'output_dims': (9,)} 32 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (6,), 'input_dims': (3,), 'output_dims': (9,)} 18 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (7,), 'input_dims': (3,), 'output_dims': (9,)} 8 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (8,), 'input_dims': (3,), 'output_dims': (9,)} 2 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (9,), 'input_dims': (3,), 'output_dims': (9,)} 0 False {'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}


In [ ]:
#|gui

sleep(2)
arc_env.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()